<a href="https://colab.research.google.com/github/ZiyueNie/FM/blob/master/src/572_hw2_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The objective is to get familiar with model calibration.
- download  today's call option prices of your favorite stock with maturity $T_1$ and several near the money strikes $(K_1, \ldots, K_n)$.
- compute IV with your bsm price engine for each option price. You are going to use USD Libor 3-month for your interest rate input. 
- plot a figure of strike v.s. IV. Do you find any volatility smile? 
- calibrate bsm volatility for the option prices, denote it by $\hat \sigma$. You may use any error function for the calibration purpose.
- reproduce option prices using your price engine with calibrated volatility, then compare how close they are to the market prices.


In [2]:
!git clone https://github.com/ZiyueNie/19ma573ZiyueNie

Cloning into '19ma573ZiyueNie'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 225 (delta 25), reused 0 (delta 0), pack-reused 155
Receiving objects: 100% (225/225), 390.05 KiB | 8.48 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [3]:
cd 19ma573ZiyueNie/src/

/content/19ma573ZiyueNie/src


In [0]:
from european_options import VanillaOption
from gbm import Gbm

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as so
import pandas as pd
import math
import scipy.stats as ss


In [28]:
def error_function(vol, gbm, option):
  gbm.vol_ratio = vol
  return np.abs(option.market_price - gbm.bsm_price(option))

def implied_volatility(gbm,option): 
  init_vol=0.1 #initial guess
  return so.fmin(error_function, init_vol, args = (gbm, option), disp = 0)[0]

def implied_volatility_list(S0,interest_rate,sigma_H,sigma_L,otype,strike_prices, maturity_time):
  gbm1= gbm.Gbm(init_state = S0,
                 drift_ratio = interest_rate,
               )
  
  option=[]
  for i in range(len(strike_prices)):
    option1= VanillaOption(
      otype = otype, # 1: 'call'
                # -1: 'put'
      strike = strike_prices[i],                
      maturity = maturity_time,
      market_price=SVM(S0,interest_rate,sigma_H,sigma_L,otype,strike_prices[i], maturity_time)
      )
    option.append(option1)

  #compute implied vols
  implied_volatility_me=[]
  for i in range(len(option)):
      implied_volatility_me.append(implied_volatility(gbm1,option[i]))
  return implied_volatility_me
gbm=Gbm()
def SVM(S0,interest_rate,sigma_H,sigma_L,otype,strike_prices, maturity_time):
    gbm1= gbm.Gbm(init_state = S0,
            drift_ratio = interest_rate,
            vol_ratio = sigma_H
               )
    gbm2= gbm.Gbm(init_state = S0,
            drift_ratio = interest_rate,
            vol_ratio = sigma_L
               )
    option1= VanillaOption(
      otype = otype, # 1: 'call'
                # -1: 'put'
      strike = strike_prices,                
      maturity = maturity_time,
      )
    return (1/2)*gbm.bsm_price(gbm1,option1)+(1/2)*gbm.bsm_price(gbm2,option1)

def delta(s0,strike_prices,r,sigma,maturity,otype):
  delta=[]
  for i in range(len(sigma)):
    d1 = (np.log(s0 / strike_prices[i]) + (r + 0.5 * sigma[i] ** 2)* maturity) / (sigma[i] * np.sqrt(maturity))
    delta.append(ss.norm.cdf(otype * d1))
  return delta

'''=================
Problem2_1
================='''
strike_price=[i for i in range(50,160,10)] #K
call_implied_volatility=implied_volatility_list(100,0.02,0.3,0.15,1,strike_price,1)
put_implied_volatility=implied_volatility_list(100,0.02,0.3,0.15,-1,strike_price,1)
plt.plot(strike_price,call_implied_volatility,label='call_implied_volatility')
plt.plot(strike_price,put_implied_volatility,label='put_implied_volatility')
plt.ylabel('implied vol')
plt.xlabel('strike')
plt.legend()
plt.show()
'''===============
Problem2_2
================='''
S0=100
r=0.02
maturity_time=1
plt.plot(S0/((math.e**(-r*maturity_time))*np.array(strike_price)),call_implied_volatility,label='Log_moneyness_call')
plt.plot(S0/((math.e**(-r*maturity_time))*np.array(strike_price)),put_implied_volatility,label='Log_moneyness_put')
plt.ylabel('implied vol')
plt.xlabel('Log_moneyness')
plt.legend()
plt.show()
'''===============
Problem1_3 call
================='''
plt.plot(delta(S0,strike_price,r,call_implied_volatility,maturity_time,1),call_implied_volatility,label='delta_call')
plt.plot(delta(S0,strike_price,r,put_implied_volatility,maturity_time,-1),put_implied_volatility,label='delta_put')
plt.ylabel('implied vol')
plt.xlabel('delta')
plt.legend()
plt.show()


AttributeError: ignored

In [0]:
'''==============
below are from the market data for underlying process
================='''
#Int_rates=2.73263/100.0   #3 month US Dollar LIBOR interest rate
gbm1 = Gbm(
    init_state =100, #today's starbucks stock price
    drift_ratio =0.02, 
    vol_ratio= 0.1
    #vol_ratio2= 0.3 #initial guess
)

In [0]:
def bsm_price1(self, vanilla_option):
    s0 = self.init_state
    sigma1 = 0.15
    sigma2 = 0.3
    r = self.drift_ratio
    
    otype = vanilla_option.otype
    k = vanilla_option.strike
    maturity = vanilla_option.maturity
    
    d1 = (np.log(s0 / k) + (r + 0.5 * sigma1 ** 2) 
          * maturity) / (sigma1 * np.sqrt(maturity))
    d2 = d1 - sigma1 * np.sqrt(maturity)
    d3 = (np.log(s0 / k) + (r + 0.5 * sigma2 ** 2) 
          * maturity) / (sigma2 * np.sqrt(maturity))
    d4 = d1 - sigma2 * np.sqrt(maturity)
    
    return ((1/2)*(otype * s0 * ss.norm.cdf(otype * d1) #line break needs parenthesis
            - otype * np.exp(-r * maturity) * k * ss.norm.cdf(otype * d2)+otype * s0 * ss.norm.cdf(otype * d3) #line break needs parenthesis
            - otype * np.exp(-r * maturity) * k * ss.norm.cdf(otype * d4)))

Gbm.bsm_price1 = bsm_price1

In [20]:
gbm1 = Gbm()
option1 = VanillaOption(strike=np.linspace(50,150,11))
print('>>>>>>>>>>call value is ' + str(gbm1.bsm_price1(option1)))
option2 = VanillaOption(otype=-1,strike=np.linspace(50,150,11))
print('>>>>>>>>>>put value is ' + str(gbm1.bsm_price1(option2)))
print(gbm1.bsm_price1(option1))
gbm1.bsm_price1(option1)

>>>>>>>>>>call value is [52.09892219 41.68232056 30.12145298 18.66452208 11.15854835  9.69734855
 11.4214534  12.45245615 11.5459406   9.44191312  7.1241268 ]
>>>>>>>>>>put value is [-0.22055415 -1.10105105 -3.1258139  -5.04664007 -3.01650907  5.05839586
 16.31860544 26.88571293 35.51530211 42.94737936 50.16569777]
[52.09892219 41.68232056 30.12145298 18.66452208 11.15854835  9.69734855
 11.4214534  12.45245615 11.5459406   9.44191312  7.1241268 ]


array([52.09892219, 41.68232056, 30.12145298, 18.66452208, 11.15854835,
        9.69734855, 11.4214534 , 12.45245615, 11.5459406 ,  9.44191312,
        7.1241268 ])

In [0]:
def bsm_price(self, vanilla_option):
    s0 = self.init_state
    sigma = self.init_state
    r = self.drift_ratio
    
    otype = vanilla_option.otype
    k = vanilla_option.strike
    maturity = vanilla_option.maturity
    
    d1 = (np.log(s0 / k) + (r + 0.5 * sigma ** 2) 
          * maturity) / (sigma * np.sqrt(maturity))
    d2 = d1 - sigma * np.sqrt(maturity)
    
    return (otype * s0 * ss.norm.cdf(otype * d1) #line break needs parenthesis
            - otype * np.exp(-r * maturity) * k * ss.norm.cdf(otype * d2))

Gbm.bsm_price = bsm_price

In [10]:
'''gbm2 = Gbm()
option1 = VanillaOption(strike=np.linspace(50,150,11))
print('>>>>>>>>>>call value is ' + str(gbm2.bsm_price(option1)))
option2 = VanillaOption(otype=-1,strike=np.linspace(50,150,11))
print('>>>>>>>>>>put value is ' + str(gbm2.bsm_price(option2)))'''

"gbm2 = Gbm()\noption1 = VanillaOption(strike=np.linspace(50,150,11))\nprint('>>>>>>>>>>call value is ' + str(gbm2.bsm_price(option1)))\noption2 = VanillaOption(otype=-1,strike=np.linspace(50,150,11))\nprint('>>>>>>>>>>put value is ' + str(gbm2.bsm_price(option2)))"

In [26]:
'''================
define an error function
===================='''
def error_function(vol, gbm, option):
  gbm.vol_ratio = vol
  return np.abs(gbm1.bsm_price(option)-gbm1.bsm_price1(option1)) #bsm_price is call price? what is market_price

'''==========
define a method to seek for an implied volatility
============'''

def implied_volatility(gbm, option):
  init_vol = .1 #initial guess
  return so.fmin(error_function, init_vol, args = (gbm, option), disp = 0)[0]
np.abs(gbm1.bsm_price1(option1) - gbm1.bsm_price(option1))
gbm1.bsm_price(option1)


In [0]:
option_list = []
option1= VanillaOption(
      otype = 1,
     strike = np.linspace(50,150,11), 
      maturity = 1,
      #market_price=call_option_data1['Last Price'][i] 
      )
option_list.append(option1)

In [15]:
data3=[]
aa=np.array(gbm1.bsm_price1(option1))
for i in range(len(aa)):
  a = implied_volatility(gbm1, option_list[i])
  data3.append(a)
print(data3)
#data2=data2.append(data3)'''
#data3=implied_volatility(gbm1,option_list)

aa


TypeError: ignored

In [0]:
plt.plot(call_option_data1['Strike'],data3,label='call implied volatility')
plt.ylabel('implied vol')
plt.xlabel('strike')
plt.legend()

In [0]:
#data4=[]
#for i in range(len(call_option_data1['Strike'])):
c=call_option_data1['Strike']
#c=np.array(c)
data4 = np.log(67.58/(c*np.exp(-2.73263/100.0*T_1)))
#data4.append(b)
#print(c)
#print(data4)
data4  #type(c) shift+enter


In [0]:
plt.plot(data4,data3,label='call implied volatility')
plt.ylabel('implied vol')
plt.xlabel('moneyness')
plt.legend()

In [0]:
d1 = (np.log(67.58 / c) + (2.73263/100.0 + 0.5 * 0.06296875000000007 ** 2) 
          * T_1) / (0.06296875000000007 * np.sqrt(T_1)) #0.06296875000000007 is calibrated volatility
data5=ss.norm.cdf(d1)
data5

In [0]:
plt.plot(data5,data3,label='call implied volatility')
plt.ylabel('implied vol')
plt.xlabel('delta')
plt.legend()

In [0]:

'''==========
define a method for error function
============'''

def error_function(vol, gbm, option_list):
  gbm.vol_ratio = vol
  err1 = 0
  for i in np.arange(len(option_list)):
    err1 = err1 + ((option_list[i]).market_price - gbm.bsm_price(option_list[i]))**2    
  return err1

In [0]:
'''==========
define a method to seek for a calibrated volatility
============'''

def bsm_calibration(gbm, option_list):
  init_vol = .1 #initial guess
  return so.fmin(error_function, init_vol, args = (gbm, option_list), disp = 0)[0]

In [0]:

# Take options of 2-mon maturity
filter1 = list(filter(lambda x: x.maturity == T_1, option_list))
calibrated_volatility1 = bsm_calibration(gbm1, filter1)
print('>>>>>>>> calibrated_volatility is ' + str(calibrated_volatility1))

In [0]:
gbm1.vol_ratio = calibrated_volatility1

#find calibrated option price
filter1_calibrated_price = [gbm1.bsm_price(filter1[i]) for i in range(len(filter1))]
print('>>>>>>>>> this is option price calculated from calibrated vol')
filter1_calibrated_price

In [0]:

x_co = [filter1[i].strike for i in range(len(filter1))]
y_co = [filter1[i].market_price for i in range(len(filter1))]
plt.plot(x_co, y_co, 'o', label='market price')

y_co = filter1_calibrated_price
plt.plot(x_co, y_co, label = 'calib price')

plt.ylabel('option price')
plt.xlabel('strike')
plt.legend();

In [0]:
strike